# Using Langchain

This notebook is a rough introduction to using Langchain which we looked at yesterday. Langchain is a nice interface on-top of LLMs. **It makes it easy for us to embed the user of LLMs in our code.**

You can use it in two ways:

1. You can download and score an LLM locally on the machine LangChain is running (free but slow).

2. You can offload calls to the LLM to a third-party API that's hosting a model (costs money but fast and easy).

To make this exercise easier and more fun, I made a dedicated API Key with $10-ish on it for the class to burn through so we can get quick results (should be good for at least a few thousand requests).

If you want to keep exploring this on your own in the future, you should set up a developer account and API key of your own [with OpenAI](https://platform.openai.com/api-keys) or with any of the [other LLM API backends](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/) LangChain supports.

In [ ]:
!pip install langchain langchain-openai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import getpass
import os

# Set up an OpenAI account and put your API Key here
os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

# We're going to try using the cheapest simplest model
model = ChatOpenAI(model="gpt-3.5-turbo")

# Enable caching
set_llm_cache(InMemoryCache())


We're going to work with OpenAI's gpt-3.5 model (there are better models, but for our use case this should be fine).

Here's the example usage from LangChain's docs:

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-efed2572-8e11-4c02-9ecd-685d06624a68-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

However, keep in mind that we can now use this in code. So lets abstract this idea into a function.

In [ ]:
def translate_to_italian(english_content):
    instruction = "Translate the following from English into Italian"
    messages = [
        SystemMessage(content=instruction),
        HumanMessage(content=english_content),
    ]
    res = model.invoke(messages)
    return res.content

translate_to_italian("This is now a function call")

'Questa è ora una chiamata di funzione'

You can imagine some pretty powerful applications for this when we work in data manipulation tools we've learned in this class such as pandas.

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/My Drive/datasets/songs.csv')

# Limit it to just 5 songs to minimize calls
taylor_df = df[df["Artist"] == "Taylor Swift"].head(5)

taylor_df["Lyrics in Italian"] = taylor_df["Lyrics"].apply(translate_to_italian)

taylor_df

Mounted at /content/gdrive


,Artist,Title,Lyrics,Lyrics in Italian
0,Taylor Swift,cardigan,"Vintage tee, brand new phone\nHigh heels on co...","Maglietta vintage, telefono nuovo di zecca\nTa..."
1,Taylor Swift,exile,"I can see you standing, honey\nWith his arms a...","Posso vederti lì in piedi, tesoro\nCon le sue ..."
2,Taylor Swift,Lover,We could leave the Christmas lights up 'til Ja...,Potremmo lasciare le luci di Natale accese fin...
3,Taylor Swift,the 1,"I'm doing good, I'm on some new shit\nBeen say...","Sto bene, sto facendo delle nuove cose\nSto di..."
4,Taylor Swift,Look What You Made Me Do,I don't like your little games\nDon't like you...,Non mi piacciono i tuoi piccoli giochi\nNon mi...


A more interesting application, in my opinion, would be to use it to handle input. Imagine, for example, our Text Adventure Game from the first week.

I've added a langchain-powered function called `match_strings`. I'm going to use it to fuzzy-match the user's input to one of the valid actions.

Try running the game below but being more loose with the way you describe you actions (the Zork developers would have killed for this).

In [ ]:
def match_input_to_valid_action(valid_actions, user_input):
    instruction = f"Rewrite the user's action as one of the following valid actions: {valid_actions} or 'None' if none are a close match."
    messages = [
        SystemMessage(content=instruction),
        HumanMessage(content=user_input),
    ]
    res = model.invoke(messages)
    msg = res.content.strip().strip("'").strip('"') # strip whitespace and quotes
    return msg

def input_action(valid_actions):
    while True:
        print("You can: ")
        for action in valid_actions:
            print(f"- {action}")
        user_input_action = input("> ")
        understood_action = match_input_to_valid_action(
            valid_actions,
            user_input_action,
        )
        if understood_action in valid_actions:
            print(f"Doing... {understood_action}\n")
            return understood_action
        else:
            print(f"You can't do '{user_input_action}' understood as '{understood_action}'")

def hallway():
    print("You are in the hallway of Mudd, you see the door to 825 and a mysterious door at the end of the hallway.")
    valid_actions = ["enter room 825", "enter mystery door"]
    action = input_action(valid_actions)
    if action == "enter 825":
        mudd_825()
    elif action == "enter mystery door":
        print("You entered the mystery room, you win!")

def mudd_825():
    print("You are in Mudd room 825. You see a door on the left and windows to the right.")
    valid_actions = ["exit door", "look out windows"]
    action = input_action(valid_actions)
    if action == "exit door":
        hallway()
    elif action == "look out windows":
        print("You look out the windows, it's a beautiful summer day")
        mudd_825()

print("Welcome to Dork! The Text Adventure Game!")
mudd_825()

Welcome to Dork! The Text Adventure Game!
You are in Mudd room 825. You see a door on the left and windows to the right.
You can: 
- exit door
- look out windows
> inspect the windows
Doing... look out windows

You look out the windows, it's a beautiful summer day
You are in Mudd room 825. You see a door on the left and windows to the right.
You can: 
- exit door
- look out windows
> leave the room
You can't do 'leave the room' understood as 'None'
You can: 
- exit door
- look out windows
> walk out
You can't do 'walk out' understood as 'None'
You can: 
- exit door
- look out windows
> walk out door
Doing... exit door

You are in the hallway of Mudd, you see the door to 825 and a mysterious door at the end of the hallway.
You can: 
- enter room 825
- enter mystery door
> go to door
You can't do 'go to door' understood as 'None'
You can: 
- enter room 825
- enter mystery door
> enter door
Doing... enter mystery door

You entered the mystery room, you win!


# Assignment

Given the above ideas, go back to your text adventure game assignment, copy and past it below, and try improving it. Feel free to use my user input function, but also try thinking of interesting things you could do with user input and abstract intructions.